In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, DistilBertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("/content/X_data_stemmed.csv", encoding = 'utf-8-sig')
data

,Unnamed: 0,0
0,0,آرتمیس دازده ساله با کمک محافظ شخص فق ماهر بات...
1,1,کرده بد در مسیر ماجخط ساحل تاالب میراند جاییکه...
2,2,به رابطه دستان که آدامسن دا شر میکرد دامن نزد ...
3,3,بد زن پنجاه پنج کیل اگر چیز را پنجبار لمس می...
4,4,مال چند سال پیشه فل با تیک صب یک از سم را زمین...
...,...,...
1376,1376,نارنیا پاسخ داده خاهد شد انتقا تیسراک بس سخ...
1377,1377,خندیدن کردند البته نم تانستند جل خد را بگیرند ...
1378,1378,تجیز کرد فراه ساز همه گنه سایل راحت که ض کنن ا...
1379,1379,اسـبپـسـرکا زندگ پیشتاز خدا خدا م کردند دباره ...


In [4]:
y_data = pd.read_csv("/content/y_data.csv", encoding = 'utf-8-sig')
y_data

,Unnamed: 0,Author,Author_ID
0,0,Artemis Fowl,1
1,1,Artemis Fowl,1
2,2,Artemis Fowl,1
3,3,Artemis Fowl,1
4,4,Artemis Fowl,1
...,...,...,...
1376,1376,c.s.lewis,10
1377,1377,c.s.lewis,10
1378,1378,c.s.lewis,10
1379,1379,c.s.lewis,10


In [ ]:
type(data['Text'])

In [5]:
texts = list(data['0'].values)  # List of input texts
labels = list((y_data['Author_ID']-1).values)  # List of corresponding labels

In [6]:
# Step 2: Load pre-trained BERT model and tokenizer
model_name = 'HooshvareLab/bert-fa-base-uncased'
# HooshvareLab/bert-fa-base-uncased
# tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=10)
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=10)
# model = RobertaForSequenceClassification.from_pretrained('distilroberta-base', num_labels=10)
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=10)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Step 3: Tokenize your data
max_length = 128  # Maximum sequence length for BERT

inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
labels = torch.tensor(labels)

In [8]:
# Step 1.1: Split the data into training, validation, and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, stratify=y_data['Author_ID'], random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels,stratify= train_labels, test_size=0.2, random_state=42)

# Step 3.1: Tokenize your data for training, validation, and testing sets
train_inputs = tokenizer(train_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
val_inputs = tokenizer(val_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
test_inputs = tokenizer(test_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

# Convert labels to tensors
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

<ipython-input-8-ece725cb7512>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
<ipython-input-8-ece725cb7512>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_labels = torch.tensor(val_labels)
<ipython-input-8-ece725cb7512>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_labels = torch.tensor(test_labels)


In [ ]:
# Step 4: Fine-tune BERT for classification
optimizer = AdamW(model.parameters(), lr=1e-5)
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

epochs = 3
for epoch in range(epochs):
    print('epoch:' , epoch)
    model.train()
    for batch in train_loader:
        print('batch' , batch)
        input_ids, attention_mask, batch_labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=batch_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


epoch: 0
batch [tensor([[     0,  43232,    175,  ...,   1359,   5184,      2],
        [     0,    746,  21643,  ...,    870,    403,      2],
        [     0, 199626,    175,  ...,    175,  18710,      2],
        ...,
        [     0,   7147,   7012,  ...,  36574,    926,      2],
        [     0,    406,   7764,  ...,  59707,  19803,      2],
        [     0,   1689,   1287,  ...,   8755,   5322,      2]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), tensor([7, 3, 7, 5, 5, 7, 6, 5, 3, 2, 2, 1, 4, 0, 6, 1, 4, 0, 1, 2, 1, 2, 4, 4,
        9, 2, 3, 6, 6, 0, 4, 5])]
batch [tensor([[     0,   1706,  21333,  ...,      6, 163664,      2],
        [     0,  31388,   3239,  ...,   8755,  12599,      2],
        [     0,    498,    746,  ...,    403, 105702,      2],
        ...,
        [     0,  11358,    406,  ...,  

In [9]:
# Step 5: Evaluate your model
model.eval()
val_dataset = TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], val_labels)
val_loader = DataLoader(val_dataset, batch_size=32)
predictions = []
true_labels = []

for batch in val_loader:
    input_ids, attention_mask, batch_labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    _, predicted_labels = torch.max(logits, dim=1)
    predictions.extend(predicted_labels.cpu().numpy())
    true_labels.extend(batch_labels.cpu().numpy())

In [10]:
# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print("Validation Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(true_labels, predictions))

# Step 6: Inference
# Make predictions on new, unseen data
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_labels)
test_loader = DataLoader(test_dataset, batch_size=32)
predictions = []
true_labels = []

for batch in test_loader:
    input_ids, attention_mask, batch_labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    _, predicted_labels = torch.max(logits, dim=1)
    predictions.extend(predicted_labels.cpu().numpy())
    true_labels.extend(batch_labels.cpu().numpy())

# Calculate evaluation metrics for test set
accuracy = accuracy_score(true_labels, predictions)
print("Test Accuracy:", accuracy)
print("Classification Report for Test Set:")
print(classification_report(true_labels, predictions))

Validation Accuracy: 0.18552036199095023
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        25
           1       0.00      0.00      0.00        17
           2       0.19      1.00      0.32        41
           3       0.00      0.00      0.00        21
           4       0.00      0.00      0.00        28
           5       0.00      0.00      0.00        37
           6       0.00      0.00      0.00        12
           7       0.00      0.00      0.00        15
           8       0.00      0.00      0.00         9
           9       0.00      0.00      0.00        16

    accuracy                           0.19       221
   macro avg       0.02      0.10      0.03       221
weighted avg       0.04      0.19      0.06       221



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Accuracy: 0.19133574007220217
Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        32
           1       0.00      0.00      0.00        22
           2       0.19      1.00      0.32        52
           3       0.00      0.00      0.00        26
           4       0.00      0.00      0.00        35
           5       0.12      0.02      0.04        46
           6       0.00      0.00      0.00        15
           7       0.00      0.00      0.00        18
           8       0.00      0.00      0.00        11
           9       0.00      0.00      0.00        20

    accuracy                           0.19       277
   macro avg       0.03      0.10      0.04       277
weighted avg       0.06      0.19      0.07       277



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
